In [15]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [2]:
def show_matrix(M, r=2):
    M = [[str(round(j,r)) for j in i] for i in M]
    for i in M:
        print("\t".join(list(i)))
        
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [3]:
X = np.array([1,2,3,6,7,8,9]) #X1 ~ X7 
labeled_i = [0,4,6] ## labeled x index
yl = np.array([0,1,1]) #Y1 & Y5 given
yl_temp = dict(zip(labeled_i, yl))

xl = np.array([x for i,x in enumerate(X) if i in labeled_i])
xu = np.array([x for i,x in enumerate(X) if not(i in labeled_i)])
E = [(0,1),(1,2),(2,3),(3,4),(4,5),(5,6)]

sigma = 1

In [4]:
## Create Matrix W
## w_ij = exp(-abs(x[i]-x[j])) if (i,j) element of E
W = []
weight_func = lambda xi, xj, sigm: np.float16(np.exp(-(np.abs(xi - xj))/sigm))
for i,x in enumerate(X):
    w = [weight_func(X[i], X[j], sigma) if ((i,j) in E or (j,i) in E) else 0 for j in range(len(X))]
    W.append(w)
W = np.array(W)

A_ = []
for i,x in enumerate(X):
    a = [W[i][j] if i!=j else -np.sum(W[i]) for j in range(len(X))]
    A_.append(a)
A_ = np.array(A_)

## Creater Matrix A
A = []
for i,x in enumerate(X):
    if i in labeled_i:
        a = [1 if j==i else 0 for j in range(len(X))]
    else:
        a = [x for x in A_[i]]
    A.append(a)
A = np.array(A)

## Create b
b = np.array([0 if i not in labeled_i else yl_temp[i] for i,x in enumerate(X)]).T

In [6]:
show_matrix(A)

1.0	0.0	0.0	0.0	0.0	0.0	0.0
0.37	-0.74	0.37	0.0	0.0	0.0	0.0
0.0	0.37	-0.42	0.05	0.0	0.0	0.0
0.0	0.0	0.05	-0.42	0.37	0.0	0.0
0.0	0.0	0.0	0.0	1.0	0.0	0.0
0.0	0.0	0.0	0.0	0.37	-0.74	0.37
0.0	0.0	0.0	0.0	0.0	0.0	1.0


## 1) Direct Solution (inverse matrix using numpy)
$u=A^{-1}b$

In [252]:
U = np.dot(np.linalg.inv(A),b)
Y = np.array([1 if u>0.5 else 0 for u in list(U)])
print("X : ", X)
print("Y : ", Y)
print("U : ", U)

X :  [1 2 3 6 7 8 9]
Y :  [0 0 0 1 1 1 1]
U :  [0.         0.09622987 0.19245973 0.90377013 1.         1.
 1.        ]


## 2) Gauss Siedel Method

In [10]:
def gs_method(A,b,x=None, max_i=1000):
    n = len(b)
    if not x:
        x = np.ones(n)
    r0 = np.dot(x,x)
    for it in range(max_i):
        for i in range(n):
            sig = 0
            for j in range(n):
                if j != i:
                    sig += A[i][j]*x[j]
            x[i] = (1/A[i][i])*(b[i] - sig)
        r1 = np.dot(x,x)
        err = abs(r0 - r1)
        r0 = r1
        if err < 1e-9:
            print("iteration ", it)
            return x
        
U = gs_method(A, b)
Y = np.array([1 if u>0.5 else 0 for u in list(U)])
print("X : ", X)
print("Y : ", Y)
print("U : ", np.round(U,3))

iteration  25
X :  [1 2 3 6 7 8 9]
Y :  [0 0 0 1 1 1 1]
U :  [0.    0.096 0.192 0.904 1.    1.    1.   ]


## 3) CG Method

In [245]:
## Choose very large K
K = 1E3
g = (1/K) * b

## Create Matrix B
B = []
for i,x in enumerate(X):
    if i in labeled_i:
        a = [1 if j==i else 0 for j in range(len(X))]
    else:
        a = [0 for x in range(len(X))]
    B.append(a)
B = np.array(B)
print("g : ")
print(g)
print("Matrix B :")
show_matrix(B,4)

g : 
[0.    0.    0.    0.    0.001 0.    0.001]
Matrix B :
1	0	0	0	0	0	0
0	0	0	0	0	0	0
0	0	0	0	0	0	0
0	0	0	0	0	0	0
0	0	0	0	1	0	0
0	0	0	0	0	0	0
0	0	0	0	0	0	1


In [246]:
def conjugate_grad(A, b, x=None):
    n = len(b)
    if not x:
        x = np.ones(n)
    r = b - np.dot(A, x)
    p = r
    rsold = np.dot(r, r)
    for i in range(n):
        Ap = np.dot(A, p)
        alpha = rsold / np.dot(p.T, Ap)
        x += alpha * p
        r -= alpha * Ap
        rsnew = np.dot(r, r)
        if np.sqrt(rsnew) < 1e-10:
            print('Itr:', i)
            break
        p = (rsnew / rsold) * p + r
    return x

U = conjugate_grad(B, g)
Y = np.array([1 if u>0.5 else 0 for u in list(U)])
print("X : ", X)
print("Y : ", Y)
print("U : ", U)

Itr: 0
X :  [1 2 3 6 7 8 9]
Y :  [0 1 1 1 0 1 0]
U :  [0.    1.    1.    1.    0.001 1.    0.001]
